# Driver notebook

This is an auto-generated notebook created by an AI Playground export. We generated three notebooks in the same folder:
- [agent]($./agent): contains the code to build the agent.
- [config.yml]($./config.yml): contains the configurations.
- [**driver**]($./driver): logs, evaluate, registers, and deploys the agent.

This notebook uses Mosaic AI Agent Framework ([AWS](https://docs.databricks.com/en/generative-ai/retrieval-augmented-generation.html) | [Azure](https://learn.microsoft.com/en-us/azure/databricks/generative-ai/retrieval-augmented-generation)) to deploy the agent defined in the [agent]($./agent) notebook. The notebook does the following:
1. Logs the agent to MLflow
2. Evaluate the agent with Agent Evaluation
3. Registers the agent to Unity Catalog
4. Deploys the agent to a Model Serving endpoint

## Prerequisities

- Address all `TODO`s in this notebook.
- Review the contents of [config.yml]($./config.yml) as it defines the tools available to your agent, the LLM endpoint, and the agent prompt.
- Review and run the [agent]($./agent) notebook in this folder to view the agent's code, iterate on the code, and test outputs.

## Next steps

After your agent is deployed, you can chat with it in AI playground to perform additional checks, share it with SMEs in your organization for feedback, or embed it in a production application. See docs ([AWS](https://docs.databricks.com/en/generative-ai/deploy-agent.html) | [Azure](https://learn.microsoft.com/en-us/azure/databricks/generative-ai/deploy-agent)) for details

In [0]:
%pip install -U -qqqq databricks-agents mlflow langchain==0.2.16 langgraph-checkpoint==1.0.12  langchain_core langchain-community==0.2.16 langgraph==0.2.16 pydantic langchain_databricks
dbutils.library.restartPython()

### Log the `agent` as an MLflow model
Log the agent as code from the [agent]($./agent) notebook. See [MLflow - Models from Code](https://mlflow.org/docs/latest/models.html#models-from-code).

In [0]:
# Log the model to MLflow
import os
import mlflow

input_example = {
    "messages": [
        {
            "role": "user",
            "content": "write the code for a tool which performs mathematical computation"
        }
    ]
}

with mlflow.start_run():
    logged_agent_info = mlflow.langchain.log_model(
        lc_model=os.path.join(
            os.getcwd(),
            'agent',
        ),
        pip_requirements=[
            "langchain==0.2.16",
            "langchain-community==0.2.16",
            "langgraph-checkpoint==1.0.12",
            "langgraph==0.2.16",
            "pydantic",
            "langchain_databricks", # used for the retriever tool
        ],
        model_config="config.yml",
        artifact_path='agent',
        input_example=input_example,
    )

## Evaluate the agent with [Agent Evaluation](https://learn.microsoft.com/azure/databricks/generative-ai/agent-evaluation/)

You can edit the requests or expected responses in your evaluation dataset and run evaluation as you iterate your agent, leveraging mlflow to track the computed quality metrics.

In [0]:
import pandas as pd

eval_examples = [
    {
        "request": {
            "messages": [
                {
                    "role": "system",
                    "content": "function calling : write the code for a tool which does mathematical computation . Sample code as below :\n{  \n  \"name\": \"generate_contextual_response\",  \n  \"description\": \"Generate a contextual response based on user query and retrieved data using a Large Language Model (LLM)\",  \n  \"parameters\": {  \n    \"type\": \"object\",  \n    \"properties\": {  \n      \"user_query\": {  \n        \"type\": \"string\",  \n        \"description\": \"The original query from the user\"  \n      },  \n      \"retrieved_data\": {  \n        \"type\": \"string\",  \n         \"description\": \"The data retrieved from the Databricks search index\"  \n      }  \n    },  \n    \"required\": [  \n      \"user_query\",  \n      \"retrieved_data\"  \n    ]  \n  }  \n}  "
                },
                {
                    "role": "user",
                    "content": "write the code for a tool which performs mathematical computation"
                }
            ]
        },
        "expected_response": None
    },
    {
        "request": {
            "messages": [
                {
                    "role": "system",
                    "content": "function calling : write the code for a tool which does mathematical computation . Sample code as below :\n{  \n  \"name\": \"generate_contextual_response\",  \n  \"description\": \"Generate a contextual response based on user query and retrieved data using a Large Language Model (LLM)\",  \n  \"parameters\": {  \n    \"type\": \"object\",  \n    \"properties\": {  \n      \"user_query\": {  \n        \"type\": \"string\",  \n        \"description\": \"The original query from the user\"  \n      },  \n      \"retrieved_data\": {  \n        \"type\": \"string\",  \n         \"description\": \"The data retrieved from the Databricks search index\"  \n      }  \n    },  \n    \"required\": [  \n      \"user_query\",  \n      \"retrieved_data\"  \n    ]  \n  }  \n}  "
                },
                {
                    "role": "user",
                    "content": "write the code for a tool which performs mathematical computation"
                },
                {
                    "role": "assistant",
                    "content": "Here is a sample code for a tool that performs mathematical computation:\n```\n{\n  \"name\": \"math_computation\",\n  \"description\": \"Perform mathematical computation\",\n  \"parameters\": {\n    \"type\": \"object\",\n    \"properties\": {\n      \"operation\": {\n        \"type\": \"string\",\n        \"description\": \"The mathematical operation to perform (e.g. addition, subtraction, multiplication, division)\"\n      },\n      \"num1\": {\n        \"type\": \"number\",\n        \"description\": \"The first number\"\n      },\n      \"num2\": {\n        \"type\": \"number\",\n        \"description\": \"The second number\"\n      }\n    },\n    \"required\": [\n      \"operation\",\n      \"num1\",\n      \"num2\"\n    ]\n  }\n}\n```\nThis tool takes in three parameters: `operation`, `num1`, and `num2`. The `operation` parameter specifies the mathematical operation to perform, and `num1` and `num2` are the two numbers to operate on.\n\nTo call this function, you would use the following format:\n```\n<function=math_computation>{\"operation\": \"addition\", \"num1\": 2, \"num2\": 3}</function>\n```\nThis would perform the addition operation on the numbers 2 and 3, and return the result.\n\nNote that the `operation` parameter can take on different values, such as \"subtraction\", \"multiplication\", \"division\", etc. The tool would need to be implemented to handle these different operations accordingly.\n\nHere is an example implementation of the `math_computation` tool:\n```\ndef math_computation(operation, num1, num2):\n  if operation == \"addition\":\n    return num1 + num2\n  elif operation == \"subtraction\":\n    return num1 - num2\n  elif operation == \"multiplication\":\n    return num1 * num2\n  elif operation == \"division\":\n    if num2 != 0:\n      return num1 / num2\n    else:\n      raise ZeroDivisionError(\"Cannot divide by zero\")\n  else:\n    raise ValueError(\"Invalid operation\")\n```\nThis implementation uses a simple if-elif-else statement to determine which operation to perform based on the value of the `operation` parameter. It then returns the result of the operation."
                },
                {
                    "role": "user",
                    "content": "Write the code for the function to which does mathematical computation. It has follow the same json format as the sample code"
                }
            ]
        },
        "expected_response": None
    }
]

eval_dataset = pd.DataFrame(eval_examples)
display(eval_dataset)

In [0]:
import mlflow
import pandas as pd

with mlflow.start_run(run_id=logged_agent_info.run_id):
    eval_results = mlflow.evaluate(
        f"runs:/{logged_agent_info.run_id}/agent",  # replace `chain` with artifact_path that you used when calling log_model.
        data=eval_dataset,  # Your evaluation dataset
        model_type="databricks-agent",  # Enable Mosaic AI Agent Evaluation
    )

# Review the evaluation results in the MLFLow UI (see console output), or access them in place:
display(eval_results.tables['eval_results'])

## Register the model to Unity Catalog

Update the `catalog`, `schema`, and `model_name` below to register the MLflow model to Unity Catalog.

In [0]:
mlflow.set_registry_uri("databricks-uc")

# TODO: define the catalog, schema, and model name for your UC model
catalog = "main"
schema = "default"
model_name = ""
UC_MODEL_NAME = f"{catalog}.{schema}.{model_name}"

# register the model to UC
uc_registered_model_info = mlflow.register_model(model_uri=logged_agent_info.model_uri, name=UC_MODEL_NAME)

## Deploy the agent

In [0]:
from databricks import agents

# Deploy the model to the review app and a model serving endpoint
agents.deploy(UC_MODEL_NAME, uc_registered_model_info.version)